In [2]:
import zipfile
import os
import shutil

# Path to your uploaded zip file
zip_path = "data.zip"

# Target folder (always "data")
extract_folder = "data"

# If "data" already exists, remove it to avoid nesting problems
if os.path.exists(extract_folder):
    shutil.rmtree(extract_folder)

# Extract into a temporary folder
temp_folder = "temp_extract"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_folder)

# If the zip already contains a "data" folder -> move its contents up
inner_path = os.path.join(temp_folder, "data")
if os.path.exists(inner_path):
    shutil.move(inner_path, extract_folder)
    shutil.rmtree(temp_folder)  # clean up
else:
    # Otherwise, just rename temp folder to "data"
    os.rename(temp_folder, extract_folder)

print("✅ Final folder structure:", os.listdir(extract_folder))


✅ Final folder structure: ['.ipynb_checkpoints', 'begin_inventory.csv', 'end_inventory.csv', 'purchases.csv', 'purchase_prices.csv', 'sales.csv', 'vendor_invoice.csv']


In [13]:
import pandas as pd
import os
from sqlalchemy import create_engine
import logging
import time

logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s -%(message)s",
    filemode="a"
)

engine = create_engine('sqlite:///inventory.db')

def ingest_db(df, table_name, engine):
    """Load CSV as dataframe and ingest into db in chunks to avoid MemoryError"""
    try:
        df.to_sql(table_name, 
                  con=engine, 
                  if_exists='replace', 
                  index=False, 
                  chunksize=10000,  # break into batches
                  method='multi')   # insert efficiently
        print(f"{table_name} uploaded successfully ✅")
    except MemoryError:
        print(f" MemoryError on {table_name}... switching to CSV chunk mode")
        for chunk in pd.read_csv(f"data/{table_name}.csv", chunksize=50000):
            chunk.to_sql(table_name,
                         con=engine,
                         if_exists='append',
                         index=False,
                         chunksize=10000,
                         method='multi')
        print(f"{table_name} uploaded in chunks ✅")


def load_raw_data():
    '''this func will load the CSVs as dataframe and ingest into db'''
    start = time.time()
    for file in os.listdir('data'):
        if '.csv' in file:
            df=pd.read_csv('data/'+file)
            logging.info(f'Ingesting {file} in db')
            ingest_db(df,file[:-4],engine)
    end = time.time()
    total_time = (end - start)/60
    logging.info('--------------INGESTION COMPLETE-------------') 

    logging.info(f'\nTotal Time Taken: {total_time} minutes')

if __name__ == '__main__':
    load_raw_data()

begin_inventory uploaded successfully ✅
end_inventory uploaded successfully ✅
purchases uploaded successfully ✅
purchase_prices uploaded successfully ✅
sales uploaded successfully ✅
vendor_invoice uploaded successfully ✅
